# Feature selection

In [23]:
import pandas as pd
import numpy as np
data = pd.read_csv('wide_wb_2000.csv')

In [24]:
data_selection = data[['finance_crude_price_brent',
       'demand_econ_gdp_cn', 'demand_econ_gdp_usa',
       'demand_econ_gdp_eu', 'demand_refinery_capacity_global',
       'demand_econ_cpi_cn', 'demand_econ_cpi_usa', 'demand_econ_cpi_eu',
       'demand_econ_ppi_cn', 'demand_econ_ppi_usa', 'demand_econ_ppi_eu',
       'demand_econ_pce_usa', 'demand_refinery_naphtha',
       'demand_refinery_eia_gasoline', 'demand_refinery_eia_diesel',
       'demand_commodity_link_gold', 
       'supply_OPEC_prod_opec', 'supply_OPEC_capacity_remain',
       'supply_canada_prod_export_can', 'supply_crudeoil_stock_commercial',
       'supply_us_prod_prod_usa', 'supply_us_prod_export_usa',
       'supply_energy_consumption_per_unit_of_gdp_china',
       'supply_doepcrin_index', 'finance_crude_price_wti',
       'finance_naturalgas_price_hh', 'finance_stock_price_nasdq100',
       'finance_stock_price_sp500', 'finance_us_bill_usdebt2',
       'finance_us_bill_usdebt5', 'finance_us_bill_usdebt10',
       'finance_us_bill_sell2buy', 'finance_usfed_policy_usd_index',
       'finance_bank_exrate_eu', 'finance_market_emo_nymex_long',
       'finance_market_emo_nymex_short', 'finance_market_emo_nymex_long/short',
       'finance_shcomp_index', 'finance_ukx_index', 'finance_cac_index',
       'finance_hsi_index', 'finance_tpx_index', 'finance_bkr_us_equity',
       'finance_hal_us_equity', 'finance_slb_us_equity',
       'finance_market_emo_vix', 'special_event_elnino',
       'geopolitics_conflict_rublerate', 'other_oil_future_ws_westafrica',
       'other_chexus_index', 'other_tbexchna_index', 'other_ustbtot_index',
       'other_cnfrbal$_index']]

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

scaler = MinMaxScaler(feature_range=(0, 1))

def feature_select(data):
    
    X = data.iloc[:, 1:]
    Y = data.iloc[:, 0]
    names = data.columns[2:]
    
    def xgboost():
        xgboost = XGBClassifier()
        xgboost.fit(X, Y)
        scores = scaler.fit_transform(list(map(lambda x: [round(x, 4)], xgboost.feature_importances_)))
        scores = [i[0] for i in scores]
        return scores
    
    def randomForest():
        rf = RandomForestRegressor()
        rf.fit(X, Y)
        scores = scaler.fit_transform(list(map(lambda x: [round(x, 4)], rf.feature_importances_)))
        scores = [i[0] for i in scores]
        return scores
    
    # def cov():
    #     scores = [np.abs(i) for i in data.corr()['finance_crude_price_brent'][1:]]
    #     return scores 
    
    crit1, crit2 = randomForest(), xgboost()
    all_crit = list(zip(names, [(i[0]+i[1])/3 for i in list(zip(crit1, crit2))]))
    return all_crit


all_crit_new = feature_select(data_selection)

for i in sorted(all_crit_new, key= lambda x:x[1]):
    print(i)

/home/bdadmin/anaconda3/envs/tft/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/bdadmin/anaconda3/envs/tft/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[02:27:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
('supply_doepcrin_index', 0.0)
('demand_econ_gdp_usa', 0.18398269497822797)
('finance_crude_price_wti', 0.1873088834644193)
('finance_shcomp_index', 0.18839113271407967)
('demand_commodity_link_gold', 0.18947338196374008)
('demand_econ_cpi_cn', 0.1905159108447955)
('finance_us_bill_sell2buy', 0.1905159108447955)
('finance_us_bill_usdebt10', 0.19163788046306093)
('other_oil_future_ws_westafrica', 0.19163788046306093)
('supply_OPEC_prod_opec', 0.19167760083166596)
('finance_stock_price_nasdq100', 0.191796761937481)
('finance_cac_index', 0.19384209933098676)
('finance_us_bill_usdebt2', 0.19488462821204214)
('finance_bank_exrate_eu', 0.1949243485806472)
('supply_us_prod_export_usa', 0.1949640689492522)
('finance_stock_price_sp500', 0.1959271